In [63]:
import pandas as pd
import altair as alt
import numpy as np

def read_data_fix_date(file_name):
    df = pd.read_csv(file_name)
    df['Meldedatum'] = pd.to_datetime(df['Meldedatum'], format='%Y/%m/%d')
    return df

df_cases = read_data_fix_date('data_cases.csv')
df_deaths = read_data_fix_date('data_deaths.csv')
df_recovered = read_data_fix_date('data_recovered.csv')
df_cases_rolling = read_data_fix_date('data_cases_rolling.csv')
df_ctr = read_data_fix_date('data_ctr_long.csv')
df_ctr_cum = read_data_fix_date('data_ctr_cum_long.csv')
df_sta = read_data_fix_date('data_sta_long.csv')
df_sta_cum = read_data_fix_date('data_sta_cum_long.csv')

df_stats = pd.read_csv('data_stats.csv')
df_population = pd.read_csv('data_population.csv')


In [58]:
df_stats

,,Cases,Recovered,Deaths,Active
0,Total,170508,147204,7533,15771
1,New Today,933,1573,116,


In [19]:
df_cases.groupby(['Meldedatum','Bundesland']).sum().reset_index()\
                [['Meldedatum','Bundesland','AnzahlFall']]

,Meldedatum,Bundesland,AnzahlFall
0,2020-01-28,Bayern,2
1,2020-01-28,Nordrhein-Westfalen,2
2,2020-01-29,Bayern,2
3,2020-01-31,Bayern,3
4,2020-02-03,Bayern,1
...,...,...,...
1125,2020-05-11,Saarland,1
1126,2020-05-11,Sachsen,2
1127,2020-05-11,Sachsen-Anhalt,2
1128,2020-05-11,Schleswig-Holstein,4


In [18]:
df_ctr_cases = df_cases.groupby(['Meldedatum'])\
                        .sum()[['AnzahlFall','AnzahlTodesfall']]
df_ctr_cases_long = \
        pd.melt(df_ctr_cases.reset_index(), id_vars=['Meldedatum'],\
                value_vars = ['AnzahlFall','AnzahlTodesfall'],\
                var_name = 'category',\
                value_name = 'Number')

# State level data
df_sta = df_cases.groupby(['Meldedatum','Bundesland']).sum().reset_index()\
                [['Meldedatum','Bundesland','AnzahlFall','AnzahlTodesfall']]
#
df_lkr = df_cases.groupby(['Meldedatum','Landkreis']).sum().reset_index()\
                [['Meldedatum','Landkreis','AnzahlFall','AnzahlTodesfall']]                


KeyError: "['AnzahlTodesfall'] not in index"

# EDA

In [ ]:
n_cases = df_cases.loc[df_cases['NeuerFall'].isin([0,1])]['AnzahlFall'].sum()
n_cases_new = df_cases.loc[df_cases['NeuerFall'].isin([-1,1])]['AnzahlFall'].sum()
n_deaths = df_cases.loc[df_cases['NeuerTodesfall'].isin([0,1])]['AnzahlTodesfall'].sum()
n_deaths_new = df_cases.loc[df_cases['NeuerTodesfall'].isin([-1,1])]['AnzahlTodesfall'].sum()
n_recovered = df_cases.loc[df_cases['NeuGenesen'].isin([0,1])]['AnzahlGenesen'].sum()
n_recovered_new = df_cases.loc[df_cases['NeuGenesen'].isin([-1,1])]['AnzahlGenesen'].sum()
n_active = n_cases - n_deaths - n_recovered


In [ ]:
df_stats = pd.DataFrame({' ':['Total','Today'],\
                        'Cases':[n_cases, n_cases_new],\
                        'Recovered':[n_recovered, n_recovered_new],\
                        'Deaths':[n_deaths, n_deaths_new],\
                        'Active':[n_active, ' ']})
df_stats.set_index(' ')


# Plots on country level

In [44]:
alt.Chart(df_ctr)\
        .mark_bar(point=True)\
        .encode(x=alt.X('monthdate(Meldedatum):O', title='Date'),\
                y=alt.Y('mean(Number):Q', title='Cumulative Cases'),\
                color='category')\
        .properties(width=800, height=400, title='Number of Cases')

alt.Chart(...)

In [50]:
df_ctr.groupby(['Meldedatum','category']).sum()

Number
Meldedatum category        
2020-01-28 case           4
2020-01-29 case           6
2020-01-31 case           9
2020-02-03 case          10
2020-02-04 case          14
...                     ...
2020-05-09 death     177349
2020-05-10 case      177652
           death     177654
2020-05-11 case      178040
           death     178041

[158 rows x 1 columns]

In [ ]:
df_cumulative = \
    pd.melt(cases_cumulative.reset_index(), id_vars=['Meldedatum'],\
            value_vars = ['AnzahlFall','AnzahlTodesfall'],\
            var_name = 'category',\
            value_name = 'Number')
df_cumulative.Meldedatum = pd.to_datetime(df_cumulative.Meldedatum, format='%Y/%m/%d')
df_cumulative.tail()

In [ ]:
# take care of log problems
df_cumulative.loc[df_cumulative['Number']<1,'Number'] = np.nan

In [ ]:
alt.Chart(df_cumulative)\
        .mark_line(point=True)\
        .encode(x=alt.X('monthdate(Meldedatum):O', title='Date'),\
                y=alt.Y('mean(Number):Q', title='Cumulative Cases'),\
                color='category')\
        .properties(width=800, height=400, title='Number of Cases')

In [ ]:
alt.Chart(df_cumulative)\
        .mark_line(point=True)\
        .encode(x=alt.X('monthdate(Meldedatum):O', title='Date'),\
                y=alt.Y('mean(Number):Q', title='Cumulative Cases',\
                        scale=alt.Scale(type='log')), color='category')\
        .properties(width=800, height=400, title='Number of Cases')

In [ ]:
df_daily_tmp = df_cases.groupby(['Meldedatum']).sum()[['AnzahlFall','AnzahlTodesfall']]
df_daily = \
    pd.melt(df_daily_tmp.reset_index(), id_vars=['Meldedatum'],\
            value_vars = ['AnzahlFall','AnzahlTodesfall'],\
            var_name = 'category',\
            value_name = 'Number')
df_daily.head()

In [ ]:
alt.Chart(df_daily)\
        .mark_bar(point=True)\
        .encode(x=alt.X('monthdate(Meldedatum):O', title='Date'),\
                y=alt.Y('mean(Number):Q', title='Cumulative Cases'),\
                color='category')\
        .properties(width=800, height=400, title='Number of Cases')

In [ ]:
df_daily.loc[df_daily['Number']>0].Number.unique()

In [ ]:
alt.Chart(df_daily.loc[df_daily['Number']>0])\
        .mark_line(point=True)\
        .encode(x=alt.X('monthdate(Meldedatum):O', title='Date'),\
                y=alt.Y('mean(Number):Q', title='Cumulative Cases',\
                       scale=alt.Scale(type='log')),\
                color='category')\
        .properties(width=800, height=400, title='Number of Cases')

# State level plots

In [ ]:
df_cases.groupby(['Meldedatum','Bundesland']).sum()


In [ ]:
df = df_cases.groupby(['Meldedatum','Bundesland']).sum().reset_index()[['Meldedatum','Bundesland','AnzahlFall']]
df

In [ ]:
df.loc[df.AnzahlFall <1,'AnzahlFall'] = np.nan

In [ ]:
tuple(sorted(list(df['Bundesland'].unique())))

In [ ]:
alt.Chart(df.loc[df['Bundesland'].isin(['Niedersachsen','Bayern'])]).mark_line(point=True).encode(x='Meldedatum',y='AnzahlFall', color='Bundesland')

In [ ]:
df.loc[df['Bundesland'].isin(['Bremen'])]

In [ ]:
alt.Chart(df.loc[df['Bundesland'].isin(['Niedersachsen','Bremen'])])\
        .mark_bar(point=True)\
        .encode(x=alt.X('monthdate(Meldedatum):O', title='Date'),\
                y=alt.Y('mean(AnzahlFall):Q', title='Cumulative Cases'),\
                color='Bundesland')\
        .properties(width=800, height=400, title='Number of Cases')

In [ ]:
alt.Chart(df).mark_line(point=True)\
    .encode(x='Meldedatum',y = alt.Y('AnzahlFall',scale=alt.Scale(type='log')), color='Bundesland')

In [ ]:
state = 'Bayern'

In [ ]:
df_plot = df.loc[df['Bundesland']==state]

In [ ]:
alt.Chart(df_plot).mark_bar(point=True).encode(x='Meldedatum',y='AnzahlFall', color='Bundesland')

In [66]:
df_sta_cum

,Meldedatum,Bundesland,AnzahlFall,AnzahlTodesfall,AnzahlGenesen
0,2020-01-28,Bayern,2,0,2
1,2020-01-28,Nordrhein-Westfalen,2,0,2
2,2020-01-29,Bayern,4,0,4
3,2020-01-31,Bayern,7,0,7
4,2020-02-03,Bayern,8,0,8
...,...,...,...,...,...
1125,2020-05-11,Saarland,2665,144,2363
1126,2020-05-11,Sachsen,4947,190,4287
1127,2020-05-11,Sachsen-Anhalt,1648,50,1453
1128,2020-05-11,Schleswig-Holstein,2956,125,2526


In [67]:
states_sel = ['Niedersachsen','Bayern']
alt.Chart(df_sta_cum.loc[df_sta_cum['Bundesland'].isin(states_sel)])\
            .mark_line(point=True)\
            .encode(x=alt.X('monthdate(Meldedatum):O', title='Date'),\
                    y=alt.Y('mean(AnzahlTodesfall):Q', title='Cumulative Cases'),\
                    color='Bundesland',\
                    tooltip=['Bundesland','AnzahlFall'])\
            .properties(width=800, height=400, title='Number of Cases')\
            .interactive()

alt.Chart(...)

# Landkreise

In [ ]:
df_lkr = df_cases.groupby(['Meldedatum','Landkreis']).sum().reset_index()[['Meldedatum','Landkreis','AnzahlFall']]
df_lkr

In [ ]:
lkr = ['LK Celle','SK Flensburg']
df_plot = df_lkr.loc[df_lkr['Landkreis'].isin(lkr)]

In [ ]:
df_cumulative['Meldedatum'].iloc[0]

In [ ]:
alt.Chart(df_cumulative).mark_line().encode(
    alt.X('month:T', axis=alt.Axis(format='%b')),
    y='mean(Number):Q',
    color='category'
).transform_timeunit(
    month='month(Meldedatum)'
)

In [ ]:
alt.Chart(df_cumulative).mark_bar().encode(
    x='monthdate(Meldedatum):O',
    y='mean(Number):Q',
    color = 'category'
)

Display Total cases by state

In [ ]:
df_cases.groupby('Bundesland').sum()[['AnzahlFall','AnzahlTodesfall']]

In [ ]:
df

In [ ]:
def plot_cases(cat = 'Bundesland', el_list = ['Bayern']):
    if cat == 'Bundesland':
        df = df_sta
    elif cat == 'Landkreis':
        df = df_lkr
    else: 
        print('Please choose either Bundesland or Landkreis')
    c = alt.Chart(df.loc[df[cat].isin(el_list)])\
            .mark_bar(point=True)\
            .encode(x=alt.X('monthdate(Meldedatum):O', title='Date'),\
                    y=alt.Y('mean(AnzahlFall):Q', title='Cumulative Cases'),\
                    color=cat)\
            .properties(width=800, height=400, title='Number of Cases')
    c.display()
plot_cases('Landkreis',['LK Celle','Region Hannover']) 

In [ ]:
cat = 'Bundesland'
el_list = ['Niedersachsen','Bayern']

alt.Chart(df.loc[df[cat].isin(el_list)])\
        .mark_bar(point=True)\
        .encode(x=alt.X('monthdate(Meldedatum):O', title='Date'),\
                y=alt.Y('mean(AnzahlFall):Q', title='Cumulative Cases'),\
                color=cat)\
        .properties(width=800, height=400, title='Number of Cases')

# State wise cases per 100k inhabitants

In [ ]:
df_cases_rolling.head()

In [ ]:
df_lkr_roll = \
        pd.melt(df_cases_rolling, id_vars=['Meldedatum', 'Landkreis'],\
                value_vars = ['AnzahlFall100k'],\
                var_name = 'category',\
                value_name = 'Number')
df_lkr_roll

In [ ]:
df_lkr.loc[df_lkr['Landkreis']=='LK Coesfeld']

In [ ]:
df = df_lkr_roll
cat = 'Landkreis'
el_list = ['LK Coesfeld','Region Hannover']

c = alt.Chart(df.loc[df[cat].isin(el_list)])\
        .mark_bar(point=True)\
        .encode(x=alt.X('monthdate(Meldedatum):O', title='Date'),\
                y=alt.Y('mean(Number):Q', title='Cumulative Cases'),\
                color=cat)\
        .properties(width=800, height=400, title='Number of Cases')
c.display()
#df[cat]
#plot_cases(df_lkr_roll, 'Landkreis',['LK Celle','Region Hannover'], 'mean(AnzahlFall100k):Q') 

In [ ]:
df_lkr_roll

In [ ]:
def plot_cases_bar(df, cat , el_list, numeric_col, title_str):
    c = alt.Chart(df.loc[df[cat].isin(el_list)])\
            .mark_bar(point=False)\
            .encode(x=alt.X('monthdate(Meldedatum):O', title='Date'),\
                    y=alt.Y(numeric_col, title='Cases'),\
                    color=cat,\
                   tooltip = ['Landkreis','Meldedatum','Number'])\
            .properties(width=800, height=400, title=title_str)
    c.display()
plot_cases_bar(df_lkr_roll, 'Landkreis',['LK Celle','Region Hannover'],'mean(Number):Q','Rolling 7-day sum of cases per 100k' ) 

In [ ]:
def plot_cases_line(df, cat , el_list, numeric_col, title_str):
    c1 = alt.Chart(df.loc[df[cat].isin(el_list)])\
            .mark_line(point=True)\
            .encode(x=alt.X('monthdate(Meldedatum):O', title='Date'),\
                    y=alt.Y(numeric_col, title='Cases'),\
                    color=cat,\
                   tooltip = ['Landkreis','Meldedatum','Number'])
            

    data = pd.DataFrame({'a': [50]})
    c2 = alt.Chart(data).mark_rule(strokeWidth=10).encode(y='a:Q',\
                                            opacity=alt.value(0.2),
                                            color = alt.value('red'))
    (c1+c2).properties(width=800, height=400, title=title_str).display()
#    c.display()
plot_cases_line(df_lkr_roll, 'Landkreis',['LK Celle','Region Hannover','LK Coesfeld'],'mean(Number):Q','Rolling 7-day sum of cases per 100k' ) 

In [ ]:
def plot_cases_line(df, cat , el_list, numeric_col, title_str):
    c = alt.Chart(df.loc[df[cat].isin(el_list)])\
            .mark_line(point=False)\
            .encode(x=alt.X('monthdate(Meldedatum):O', title='Date'),\
                    y=alt.Y(numeric_col, title='Cases'),\
                    color=cat,\
                   tooltip = ['Landkreis','Meldedatum','Number'])\
            .properties(width=800, height=400, title=title_str).display()
            
    #c.display()
plot_cases_line(tmp, 'Landkreis',['LK Celle','Region Hannover','LK Coesfeld','Lockdown Threshold'],'mean(Number):Q','Rolling 7-day sum of cases per 100k' ) 

In [ ]:
tmp = df_lkr_roll.append({'Meldedatum':df_lkr_roll['Meldedatum'].min(), \
                    'Landkreis':'Lockdown Threshold',\
                    'category':'AnzahlFall100k',\
                    'Number': 50},ignore_index=True)
tmp = tmp.append({'Meldedatum':df_lkr_roll['Meldedatum'].max(), \
                    'Landkreis':'Lockdown Threshold',\
                    'category':'AnzahlFall100k',\
                    'Number': 50},ignore_index=True)
tmp

In [ ]:
#df_cases_pop.groupby(['Meldedatum']).sum()['AnzahlFall'].rolling(3).sum()
df_cases_rolling = df_cases_pop.groupby(['Meldedatum','Landkreis']).sum()[['Case_per_100k']].rolling(7).sum().reset_index()

In [ ]:
tmp = df_cases_pop.groupby(['Meldedatum','Landkreis']).sum()[['Case_per_100k']].reset_index()
tmp.loc[tmp['Landkreis']=='LK Coesfeld' ][['Case_per_100k']].rolling(7).sum()

In [ ]:
df2['AnzahlFall'].rolling('7d').sum()

In [ ]:
keep_col = ['AnzahlFall','AnzahlTodesfall']#,'Bundesland','Landkreis']
df3 = df_cases_pop.groupby('IdLandkreis').rolling('7d').sum()[keep_col].reset_index()
#df3.loc[df3['Landkreis']=='LK Coesfeld']

In [ ]:
df2 = df_cases.sort_values(by='Meldedatum').set_index('Meldedatum')

In [ ]:
alt.Chart(df_cases_rolling.loc[df_cases_rolling['Landkreis'].isin(['LK Coesfeld','LK Celle'])])\
        .mark_bar(point=True)\
        .encode(x=alt.X('monthdate(Meldedatum):O', title='Date'),\
                y=alt.Y('mean(Case_per_100k):Q', title='Cumulative Cases'),\
                color='Landkreis',\
                tooltip = ['Landkreis','Case_per_100k'])\
        .properties(width=800, height=400, title='Rolling number of cases per week per 100k inhabitant')

# Bar plot cases by state

In [ ]:
df_cases_pop['Meldedatum'].dtype

In [ ]:
tmp = df_cases.groupby('Bundesland').sum()[['AnzahlFall','AnzahlTodesfall']]\
        .sort_values(by='AnzahlFall', ascending=False).reset_index()
tmp = tmp.melt(id_vars='Bundesland', value_vars = ['AnzahlFall', 'AnzahlTodesfall'], value_name='Cases')
list(tmp['Bundesland'].unique())

In [ ]:
alt.Chart(tmp)\
            .mark_bar()\
            .encode(x=alt.X('Bundesland:O', title='Bundesland', sort=list(tmp['Bundesland'].unique())),\
                    y=alt.Y('Cases:Q', title='Cumulative Cases'),\
                    color='variable',\
                   tooltip=['Cases','variable'])\
            .properties(width=800, height=400, title='Number of Cases')